In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers.noise import GaussianDropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [0]:
def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

In [0]:
train_df = train.drop(['target','Modify_field','Operation_type','value_after_operation'], axis=1)
X = train_df.values

features = train_df.columns

target = train['target']
y = train['target'].values

In [0]:
nrounds=500
kfold = 5

params = {'max_bin':20, 'learning_rate' : 0.002, 'boosting_type':'gbdt',  'objective': 'regression', 'metric': 'l1', 'max_depth':10,
          'feature_fraction': 0.8, 'bagging_fraction':0.9, 'bagging_freq':20, 'min_data': 1000,'num_leaves':512}

folds = StratifiedKFold(n_splits=kfold, shuffle=False, random_state=1)
oof = np.zeros(len(train))
predictions = np.zeros(len(test_sample))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx],categorical_feature=['Product_Types','Product_id','ad_account_id','ad_id','ad_industry_id'])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx],categorical_feature=['Product_Types','Product_id','ad_account_id','ad_id','ad_industry_id')

    clf = lgb.train(params, trn_data, nrounds, valid_sets = [trn_data, val_data],feval=gini_lgb, verbose_eval=100, early_stopping_rounds = 300)

    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    predictions += clf.predict(test_sample[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(r2_score(target, oof)))